# Modelling learning systems in artificial and spiking neural networks

</br></br>

* Author: Jens Egholm Pedersen ``<xtp778@alumni.ku.dk>``
* Supervisor: Martin Elsman ``<mael@diku.dk>``

## Agenda

1. Introduction
2. Coding spikes as information
3. Domain-specific language: Volr
4. Classification experiments
5. Findings and reflections

### Introduction: Motivation

Spiking neural networks (SNN) promising, but
* Diverse and non-standard representations
* Attempts to learn still not on-par with ANN

* Relatively little attention to information representation
* Spill-over from artificial neural networks (ANN)

### Introduction: Hypotheses

1. The Volr DSL can translate into spiking and non-spiking neural networks such that the network topologies are retained.
2. Using training it is possible for spiking and non-spiking models to solve an MNIST recognition task.

## Agenda

1. ~~Introduction~~
2. Theory: Coding spikes as information
3. Domain-specific language: Volr
4. Classification experiments
5. Findings and reflections

### Theory: Artificial neural networks

* Computational graph structures

<center>
<img src="ff.png" style="width:48%"/>
<img src="bp.png" style="width:48%"/>
</center>

<span style="font-size: 70%">Source: R. Rojas: Neural Networks, Springer 1996</span>

### Theory: Electrical model of a leaky-integrate and fire (LIF) neuron

<table>
    <tr style="background: white">
        <td>$\frac{dv}{dt} = - {1 \over \tau_{c}} (V - J R)$</td>
        <td><img src="rc.png" style="width: 66%; float: right"/></td>
    </tr>
</table>

Where $V$ is the voltage difference between the inner and outer neuron membranes, $J_M$ is the input charge, $R$ is the leak resistance and $\tau_{C}$ is the membrane time constant

$J_R = {V \over R}$

<center><img src="lif.png" style="width:40%"/></center>

### Theory: LIF activations and parameters

<p style="font-size:70%">Input: Constant input current of 2 nA</p>
<img src="../report/images/membrane.png" style="width: 50%; float:left;"/>

<img src="parameters.png" style="width: 48%; float: right;"/>

### Theory: Learning in SNN

* Feedforward
  * Translation of ANN input/output
* Backpropagation
  * Similar (derivable) activation model

* Learning: spike rate coding

### Theory: Coding scheme for SNN

Given some activation potential $x$, input current $v$, voltage threshold $V_{thr}$, a max spike rate $r_{max}$, the neuron membrane potential $v$, and some error $\epsilon$, the neuron spike rate over an interval $t$ is approximately:

\begin{equation}
r_i^1(t) = x_i^1 r_{max} {V_{thr} \over V_{thr} + \epsilon_i^1 } - {v_i^1(t) \over t (V_{thr} + \epsilon_i^1) }
\end{equation}


<span style="font-size: 70%">Source: Rueckauer et al., Frontiers in Neuroscience 11:682, 2017.</span>

### Theory: Linear translation from current to spike rate

<img src="../report/images/spike_rate.png" style="width:50%"/>

### Theory: Composition of  populations

$w^l = {0.065 \over n^{l-1}}, r \in [0;38]$

![](../report/images/spike_rate_mnist.png)

### Theory: Spike rate coding and backpropagation

* Input coding with linear model
* Weight adaptation from ANN to SNN weight parameters


* Forward pass in simulator
* Backward pass with regular backpropagation

## Agenda

1. ~~Introduction~~
2. ~~Theory: Coding spikes as information~~
3. Domain-specific language: Volr
4. Classification experiments
5. Findings and reflections

### DSL: Use cases

* Model plastic neural network structures
* Comparable topologies/semantics

### DSL: Requirements
1. Semantic consistency
  * Between ANN and SNN
2. Translation to ANN and SNN
3. Learning
  * Backpropagation
4. Well-typed
  * Type checking at compile time

### DSL: Expressions

Expressions, values and types

![](dsl.png)

### DSL: Implementation

<img src="architecture.svg" style="width:35%; float:left;"/>

<img src="../report/images/volr-detailed.svg" style="width:25%; margin-left:10%; float: left;"/>


### DSL: Tests

* Unit tests
  * Compiler 
  * Futhark library
  * Python library

* Integration tests
  * Experiments

### DSL: Example topology test

<table style="font-size:100%; width:65%">
    <tr><td colspan=2><center>
  <code><b>dense</b> 2 4 ⦶ <b>dense</b> 4 2</code></center></td></tr>
    <tr>
        <td width="70%" style="font-size:60%; vertical-align: top; width=70%;">
<pre style="padding:0">...
let x0 = dense (2, 4)
let x1 = dense (4, 2)
let n = connect_layers x0 x1
...
</pre>
</td>
<td style="font-size:60%">
<pre style="padding:0">...
p1 = Population(2, ...)
p3 = Population(4, ...)
p5 = Population(2, ...)
layer0 = Dense(p1, p3)
layer1 = Dense(p3, p5)
...
</pre>
</td>
    </tr>
</table>

## Agenda

1. ~~Introduction~~
2. ~~Coding spikes as information~~
3. ~~Domain-specific language: Volr~~
4. Classification experiments
5. Findings and reflections

### Experiment: Experimental setup

* Fixed neuron parameters, excitatory weights
* 10 experiments with 80/20% training/testing split

* SNN experiments run twice: 
  * Random weights
  * Weight transfer from Futhark to NEST

### Experiment: results

Accuracies with standard deviations

| Task | ANN | SNN w/ random weights | SNN w/ transferred weights | Chance level |
| --- | --- | --- | --- | --- |
| NAND | 1.000 | 0.370 $\pm$ 0.040 | 0.69 $\pm$ 0.216 | 0.75 |
| XOR | 1.000 | 0.530 $\pm$ 0.038 | 0.585 $\pm$ 0.033 | 0.5 |
| Sequential MNIST | 0.710 | 0.147 $\pm$ 0.044 | 0.098 $\pm$ 0.006 | 0.1 |



### Experiment: NAND and XOR

<table>
    <tr><th><center>NAND</center></th><th><center>XOR</center></th></tr>
    <tr><td><img src="../report/images/nand.png"/></td>
    <td><img src="../report/images/xor.png"/></td></tr>
</table>

### Experiment: MNIST


<center><img src="../report/images/mnist_snn.png" style="width:70%"/></center>

### Experiment: Conclusions

<table>
    <tr style="background-color:white"><td width="40%">
<ul style="text-align:left">
    <li>Bad prediction scores</li>
    <li style="list-style-type:none">
        <ul><li>Imprecise transfer model</li>
    <li>Solely excitatory connections</li>
    <li>No exploration of hyperparameters</li>
        </ul></li>
    <li>Learning occurs</li>
    <li>Improvement when importing ANN weights</li>
    <li>Large number of dead neurons</li>
    <li style="list-style-type:none"><ul><li>ReLU model non-differentiable around 0</li></ul></li>
        </ul>
        </td>
        <td>
        <img src="../report/images/nand.png"/>
        <img src="../report/images/xor.png"/>
        </td>
        <td><img src="../report/images/mnist_snn.png"/></td>
    </tr>
</table>

## Agenda

1. ~~Introduction~~
2. ~~Coding spikes as information~~
3. ~~Domain-specific language: Volr~~
4. ~~Classification experiments~~
5. Findings and reflections

### Thesis findings

<table><tr>
    <th>Hypothesis</th><th>Question</th><th>Result</th></tr>
    <tr><td rowspan="2">The Volr DSL can translate into spiking and non-spiking neural networks such that the network topologies are retained</td><td>Translation to ANN</td><td>Confirmed</td>
    </tr>
    <tr><td>Translation to SNN</td><td>Confirmed</td></tr>
    <tr>
    <td rowspan="2">Using training it is possible for spiking and non-spiking models to solve an MNIST recognition task</td><td>Learning an MNIST task in ANN</td><td>Confirmed</td>
    </tr>
    <tr><td>Learning an MNIST task in SNN</td><td>Unconfirmed</td></tr>
    </table>

### Thesis contributions


 1. a domain-specific language for modelling neural network topologies
 2. a preliminary model for generalisable learning in SNN
 3. a method for transferring optimised non-spiking parameters to SNN